# Ajuste de hiperparámetros (GridSearch)

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el dataset
df = pd.read_csv("df_reduced.csv")

# Separar características y etiquetas
X = df.drop(columns=["Hz", "medida", "Tipo"])
y = df["Tipo"]

# División en entrenamiento y prueba con estratificación
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

param_grid = {
    'n_estimators': [50, 100],  # Menos estimadores
    'max_features': ['sqrt', 'log2']  # Opciones válidas para max_features
}

# Crear el modelo RandomForest
model = RandomForestClassifier(random_state=42)

# Aplicar GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Ajustar el modelo con los mejores hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Predecir en el conjunto de prueba usando el mejor modelo
y_pred = best_model.predict(X_test)

# Calcular métricas
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy (test con stratify y GridSearch): {acc:.2f}")
print("Matriz de Confusión:")
print(cm)
print("Reporte de Clasificación:")
print(report)

# Obtener todas las clases conocidas por el modelo
labels = sorted(best_model.classes_)

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, 
    annot=True,
    fmt='d', 
    cmap='Blues',
    xticklabels=labels,
    yticklabels=labels
)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix (Test con Stratify y GridSearch)")
plt.tight_layout()
plt.show()

# Imprimir los mejores parámetros encontrados
print("Mejores hiperparámetros encontrados por GridSearchCV:")
print(grid_search.best_params_)


KeyError: "['Hz', 'medida', 'Tipo'] not found in axis"

# Guardado de modelo

In [ ]:
import pickle  
with open('modelo_random_forest_best.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Modelo guardado exitosamente con pickle.")
